In [1]:
import tensorflow as tf
import random
import sys
# ----------- change the paths according to your environment -----------
sys.path.append('.')
import mypreprocessing
from generator import DataGenerator
from models import getNetwork
from custom_layers import MeanOverTime
import sys, os
import json
import scipy
import keras
import tensorflow as tf
from keras import optimizers, initializers, regularizers, constraints
from utils import *
from sklearn import metrics
import matplotlib as plt
from keras.layers import *
from tcn import TCN
from keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyp
from keras.layers import Layer
from sklearn.metrics import classification_report

class ChannelAttention(Layer):
    def __init__(self, ratio = 8):
        super(ChannelAttention, self).__init__()
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]

        self.dense = tf.keras.layers.Dense(units=self.channels // self.ratio,
                                           activation='relu',
                                           kernel_initializer='he_normal',
                                           use_bias=True)
        self.dense_output = tf.keras.layers.Dense(units=self.channels,
                                                  kernel_initializer='he_normal',
                                                  use_bias=True)

    def call(self, inputs):
        # inputs = tf.expand_dims(inputs, axis=2)
        gap = tf.reduce_mean(inputs, axis=[0, 1], keepdims=True)

        attention = self.dense(gap)
        attention = self.dense_output(attention)
        attention = tf.keras.activations.sigmoid(attention)

        output = tf.multiply(inputs, attention)

        return output

tf.random.set_seed(42)
print('OK')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

os.getcwd()

folder_process_data = './DB2'

os.environ['PYTHONHASHSEED'] = '0'

print('Keras:', keras.__version__)
print('Tensorflow:', tf.__version__)

# 1. Logging
'''
if len(sys.argv) == 4:
    CONFIG_FILE = str(sys.argv[1])
    SUBJECT = int(sys.argv[2])
    TIMESTAMP = int(sys.argv[3])
else:
    print('Expected different number of arguments. {} were given'.format(len(sys.argv) - 1))
    sys.exit()
    '''
# ----------- change the paths according to your environment -----------
CONFIG_FILE='./config_DB2/TCCNet_aot_2500.json'
import datetime
from datetime import date
SUBJECT=1
GESTURES=50
currentDateAndTime = datetime.datetime.now()

current_date = datetime.datetime.now()
TIMESTAMP = current_date .strftime("%Y%m%d%H%M%S")

# ----------- change the paths according to your environment -----------
with open(CONFIG_FILE) as json_file:
    config_params = json.load(json_file)
outdir='./Result'
LOGGING_FILE_PREFIX = config_params['logging']['log_file'] + '_' + str(TIMESTAMP)
if config_params['logging']['enable']:
    LOGGING_FILE = './Result/L_' + LOGGING_FILE_PREFIX + '.log'# 'Musa_NinaPro_Project/ProcessedData/DB1/Result/L_' + LOGGING_FILE_PREFIX + '.log'
    LOGGING_TENSORBOARD_FILE = './Result/tblogs/L_' + LOGGING_FILE_PREFIX #'Musa_NinaPro_Project/ProcessedData/DB1/Result/tblogs/L_' + LOGGING_FILE_PREFIX

if config_params['model']['save']:
    MODEL_SAVE_FILE = './Result/models/O1_' + LOGGING_FILE_PREFIX + '_{}.json'
    MODEL_WEIGHTS_SAVE_FILE = './Result/models/O2_' + LOGGING_FILE_PREFIX + '_{}.h5'

METRICS_SAVE_FILE = './Result/metrics/O3_' + LOGGING_FILE_PREFIX + '_{}.mat'

if not os.path.exists(os.path.dirname(METRICS_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(METRICS_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

if not os.path.exists(os.path.dirname(MODEL_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(MODEL_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

if not os.path.exists(os.path.dirname(LOGGING_TENSORBOARD_FILE)):
    try:
        os.makedirs(os.path.dirname(LOGGING_TENSORBOARD_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise


print('Logging file: {}'.format(LOGGING_FILE))
print('Tensorboard file: {}'.format(LOGGING_TENSORBOARD_FILE))
print('Model JSON file: {}'.format(MODEL_SAVE_FILE))
print('Model H5 file: {}'.format(MODEL_WEIGHTS_SAVE_FILE))
print('Metrics file: {}'.format(METRICS_SAVE_FILE))

# 2. Config params generator
PARAMS_TRAIN_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('train_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_TRAIN_GENERATOR[key] = params_gen[key]

PARAMS_VALID_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('valid_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_VALID_GENERATOR[key] = params_gen[key]

2025-04-15 07:52:23.101828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 07:52:24.002259: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OK
1 Physical GPUs, 1 Logical GPUs
Keras: 2.12.0
Tensorflow: 2.12.0
Logging file: ./Result/L_TCCNet_att_20250415075228.log
Tensorboard file: ./Result/tblogs/L_TCCNet_att_20250415075228
Model JSON file: ./Result/models/O1_TCCNet_att_20250415075228_{}.json
Model H5 file: ./Result/models/O2_TCCNet_att_20250415075228_{}.h5
Metrics file: ./Result/metrics/O3_TCCNet_att_20250415075228_{}.mat


2025-04-15 07:52:28.337915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 07:52:28.379552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 07:52:28.379626: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 07:52:28.388527: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:33:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 07:52:28.388599: I tensorflow/compile

In [2]:
target_subject = 1

In [3]:
# 3. Initialization
INPUT_DIRECTORY = 'DB2'
quarter_batch = True

#INPUT_DIRECTORY = folder_process_data        #link of processed dataset
PARAMS_TRAIN_GENERATOR['preprocess_function_1'] = [mypreprocessing.lpf]
PARAMS_TRAIN_GENERATOR['preprocess_function_1_extra'] = [{'fs': 100}]
PARAMS_TRAIN_GENERATOR['data_type'] = 'rms'
PARAMS_TRAIN_GENERATOR['classes'] = [i for i in range(50)]#+[0]
PARAMS_TRAIN_GENERATOR['repetitions'] = [1, 3, 4, 6]
# PARAMS_TRAIN_GENERATOR['dim'] = [1,10]
# PARAMS_TRAIN_GENERATOR['window_size'] = 200
# PARAMS_TRAIN_GENERATOR['window_step'] = 50

PARAMS_VALID_GENERATOR['preprocess_function_1'] = [mypreprocessing.lpf]
PARAMS_VALID_GENERATOR['preprocess_function_1_extra'] = [{'fs': 100}]
PARAMS_VALID_GENERATOR['data_type'] = 'rms'
PARAMS_VALID_GENERATOR['classes'] = [i for i in range(50)]#+[0]
PARAMS_VALID_GENERATOR['repetitions'] = [2, 5]
# PARAMS_VALID_GENERATOR['dim'] = [1,10]
# PARAMS_VALID_GENERATOR['window_size'] = 200
# PARAMS_VALID_GENERATOR['window_step'] = 50

if quarter_batch == True:
    PARAMS_TRAIN_GENERATOR['batch_size'] = int(64)
    PARAMS_VALID_GENERATOR['batch_size'] = int(64)
    # PARAMS_TRAIN_GENERATOR['batch_size'] = int(128/16)
    # PARAMS_VALID_GENERATOR['batch_size'] = int(128/16)
    learning_late = config_params['training']['l_rate']/8
else:
    PARAMS_TRAIN_GENERATOR['batch_size'] = int(128)
    PARAMS_VALID_GENERATOR['batch_size'] = int(128)
    learning_late = config_params['training']['l_rate']

# ---------- Change the subject number (now is 28) according to DB2 ----------
SUBJECTS = config_params['dataset'].get('subjects', [i for i in range(1, 41)])
if np.min(SUBJECTS) <= 0 or np.max(SUBJECTS) >= 41:
    raise AssertionError('Subject IDs should be between 1 and 41 inclusive for DB2. Were given {}\n'.format(SUBJECTS))

PARAMS_TRAIN_GENERATOR.pop('input_directory', '')
PARAMS_VALID_GENERATOR.pop('input_directory', '')

#MODEL = getNetwork(config_params['model']['name'])

mean_train, mean_test, mean_test_3, mean_test_5 = [], [], [], []
mean_cm = []
mean_train_loss, mean_test_loss = [], []

if config_params['logging']['enable']:
    if os.path.isfile(LOGGING_FILE) == False:
        with open(LOGGING_FILE, 'w') as f:
            f.write(
                'TIMESTAMP: {}\n'
                'KERAS: {}\n'
                'TENSORFLOW: {}\n'
                'DATASET: {}\n'
                'TRAIN_GENERATOR: {}\n'
                'VALID_GENERATOR: {}\n'
                'MODEL: {}\n'
                'MODEL_PARAMS: {}\n'
                'TRAIN_PARAMS: {}\n'.format(
                    TIMESTAMP,
                    keras.__version__, tf.__version__,
                    config_params['dataset']['name'], PARAMS_TRAIN_GENERATOR,
                    PARAMS_VALID_GENERATOR,
                    config_params['model']['name'], config_params['model']['extra'],
                    config_params['training']
                )
            )
            f.write(
                'SUBJECT,TRAIN_SHAPE,TEST_SHAPE,TRAIN_LOSS,TRAIN_ACC,TEST_LOSS,TEST_ACC,TEST_TOP_3_ACC,TEST_TOP_5_ACC\n')

print('Subject: {}'.format(target_subject))
input_dir = '{}/subject-{:02d}'.format(INPUT_DIRECTORY, target_subject)
#print(input_dir)
#print("PARAMS_TRAIN_GENERATOR")
#print(PARAMS_TRAIN_GENERATOR)

train_generator = DataGenerator(input_directory=input_dir, **PARAMS_TRAIN_GENERATOR) #classes = [0,range(13,52)],
valid_generator = DataGenerator(input_directory=input_dir, **PARAMS_VALID_GENERATOR)
X_test, Y_test, test_reps = valid_generator.get_data()                                  # X test,  Y test

# print('Train generator:')
# print(train_generator)
# print('Test generator:')
# print(valid_generator)

Subject: 1


In [4]:
import keras
import tensorflow as tf
from keras import optimizers, initializers, regularizers, constraints
from keras.layers import *
from tcn import TCN
from keras.models import Model
from keras.layers import Layer
from sklearn.metrics import classification_report, accuracy_score

class ChannelAttention(Layer):
    def __init__(self, ratio = 8):
        super(ChannelAttention, self).__init__()
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]

        self.dense = tf.keras.layers.Dense(units=self.channels // self.ratio,
                                           activation='relu',
                                           kernel_initializer='he_normal',
                                           use_bias=True)
        self.dense_output = tf.keras.layers.Dense(units=self.channels,
                                                  kernel_initializer='he_normal',
                                                  use_bias=True)

    def call(self, inputs):
        # inputs = tf.expand_dims(inputs, axis=2)
        gap = tf.reduce_mean(inputs, axis=[0, 1], keepdims=True)

        attention = self.dense(gap)
        attention = self.dense_output(attention)
        attention = tf.keras.activations.sigmoid(attention)

        output = tf.multiply(inputs, attention)

        return output

def se_block(in_block, ch, ratio=12):
    z = GlobalAveragePooling1D()(in_block)
    x = Dense(ch // ratio, activation='relu')(z)
    x = Dense(ch, activation='sigmoid')(x)
    return multiply([in_block, x])



columns = 12
gestures = 50

input_shape = (None, columns)

input = Input(shape=input_shape)

branch1 = TCN(nb_filters=32, nb_stacks=1, kernel_size=3, return_sequences=True)(input)
branch1 = Bidirectional(LSTM(64, return_sequences=True))(branch1)

branch2 = Conv1D(16, 3)(input)
branch2 = SeparableConv1D(filters=16, kernel_size=3, padding='same', activation='relu')(branch2)
branch2 = se_block(branch2, ch=16)

branch3 = Bidirectional(TCN(nb_filters=32, nb_stacks=1, kernel_size=3, return_sequences=True))(input)

# Flatten the sequences before concatenating
branch1_flat = GlobalAveragePooling1D()(branch1)
branch2_flat = GlobalAveragePooling1D()(branch2)
branch3_flat = GlobalAveragePooling1D()(branch3)
# input_flat = GlobalAveragePooling1D()(input)

# x = concatenate([tcn1_flat, tcn2_flat, tcn3_flat, input_flat])
x = concatenate([branch1_flat, branch2_flat, branch3_flat])
x = Dropout(0.2)(x)
x = ChannelAttention()(x)
output = Dense(50, trainable=True, activation='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

from tensorflow.keras.callbacks import Callback

class SavePredictionsCallback(Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        self.predictions_per_epoch = []
        self.true_labels_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test)
        self.predictions_per_epoch.append(y_pred)
        self.true_labels_per_epoch.append(self.y_test)
        print(f"Epoch {epoch+1}: saved predictions and true labels")

2025-04-15 07:52:58.068709: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-04-15 07:52:58.070414: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-04-15 07:52:58.071662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 12)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, None, 16)     592         ['input_1[0][0]']                
                                                                                                  
 separable_conv1d (SeparableCon  (None, None, 16)    320         ['conv1d[0][0]']                 
 v1D)                                                                                             
                                                                                                  
 global_average_pooling1d (Glob  (None, 16)          0           ['separable_conv1d[0][0]']   

In [5]:
if config_params['training']['optimizer'] == 'adam':
    optimizer = optimizers.Adam(learning_rate=learning_late, epsilon=0.001)

elif config_params['training']['optimizer'] == 'sgd':
    optimizer = optimizers.SGD(learning_rate=learning_late, momentum=0.9)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', top_3_accuracy, top_5_accuracy])

train_callbacks = []

lrScheduler = MyLRScheduler(**config_params['training']['l_rate_schedule'])
train_callbacks.append(lrScheduler)
#print(train_generator)

save_predictions_callback = SavePredictionsCallback(X_test, Y_test)

history = model.fit(train_generator, epochs=2,
                              validation_data=(X_test,Y_test), callbacks=[save_predictions_callback], verbose=2)

Y_pred = model.predict(X_test)

y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(Y_test, axis=1)

#print(history.history)

#acc_list.append(max(history.history['accuracy']))

if config_params['model']['save']:
    # serialize model to JSON
    model_json = model.to_json()
    with open(MODEL_SAVE_FILE.format(SUBJECT), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(MODEL_WEIGHTS_SAVE_FILE.format(SUBJECT))
    print("Saved model to disk")

# Confusion Matrix
# C_{i, j} is equal to the number of observations known to be in group i but predicted to be in group j.
cnf_matrix_frame = metrics.confusion_matrix(y_test, y_pred)
if np.array(mean_cm).shape != cnf_matrix_frame.shape:
    mean_cm = cnf_matrix_frame
else:
    mean_cm += cnf_matrix_frame

#dict_keys(['loss', 'accuracy', 'top_3_accuracy', 'top_5_accuracy', 'val_loss', 'val_accuracy', 'val_top_3_accuracy', 'val_top_5_accuracy'])

mean_train.append(history.history['accuracy'][-1])
mean_test.append(history.history['val_accuracy'][-1])
mean_train_loss.append(history.history['loss'][-1])
mean_test_loss.append(history.history['val_loss'][-1])
mean_test_3.append(history.history['val_top_3_accuracy'][-1])
mean_test_5.append(history.history['val_top_5_accuracy'][-1])

if config_params['logging']['enable']:
    with open(LOGGING_FILE, 'a') as f:
        f.write('{},{},{},{},{},{},{},{},{}\n'.format(SUBJECT, train_generator.__len__() * PARAMS_TRAIN_GENERATOR['batch_size'], valid_generator.__len__(),
            mean_train_loss[-1], mean_train[-1], mean_test_loss[-1], mean_test[-1], mean_test_3[-1], mean_test_5[-1]))

metrics_dict = {
    'mean_cm': mean_cm,
    'mean_test': mean_test,
    'mean_test_3': mean_test_3,
    'mean_test_5': mean_test_5,
    'mean_train': mean_train,
    'mean_train_loss': mean_train_loss,
    'mean_test_loss': mean_test_loss
}
scipy.io.savemat(METRICS_SAVE_FILE.format(SUBJECT), metrics_dict)

Epoch 1/2


2025-04-15 07:52:59.471657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2025-04-15 07:53:00.629125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-04-15 07:53:00.630946: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

2/2 [==============================] - 2s 507ms/step
Epoch 1: saved predictions and true labels
40/40 - 76s - loss: 12.8092 - accuracy: 0.0195 - top_3_accuracy: 0.0664 - top_5_accuracy: 0.1078 - val_loss: 3.7637 - val_accuracy: 0.0469 - val_top_3_accuracy: 0.1406 - val_top_5_accuracy: 0.2500 - 76s/epoch - 2s/step
Epoch 2/2
2/2 [==============================] - 1s 442ms/step
Epoch 2: saved predictions and true labels
40/40 - 59s - loss: 3.9681 - accuracy: 0.0324 - top_3_accuracy: 0.0949 - top_5_accuracy: 0.1559 - val_loss: 3.6593 - val_accuracy: 0.0781 - val_top_3_accuracy: 0.2031 - val_top_5_accuracy: 0.3125 - 59s/epoch - 1s/step
2/2 [==============================] - 1s 477ms/step
Saved model to disk


In [6]:
print("-----subject-------")
print(target_subject)
print("----accuracy-----")
print(max(history.history['accuracy']))
print("---val accuracy---")
print(max(history.history['val_accuracy']))

predictions_per_epoch = save_predictions_callback.predictions_per_epoch
true_labels_per_epoch = save_predictions_callback.true_labels_per_epoch

for epoch, (y_pred, y_true) in enumerate(zip(predictions_per_epoch, true_labels_per_epoch)):
    y_pred = np.argmax(y_pred, axis=1)
    if (round(metrics.accuracy_score(y_test, y_pred),4) == round(max(history.history['val_accuracy']),4)):
        print(f"Epoch {epoch+1}")
        # print("Predictions:", y_pred)
        # print("True Labels:", y_true)

        print(classification_report(y_test, y_pred, digits=4))

# target_subject += 1

-----subject-------
1
----accuracy-----
0.03242187574505806
---val accuracy---
0.078125
Epoch 2
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000         2
           2     0.0000    0.0000    0.0000         2
           3     0.0000    0.0000    0.0000         2
           4     0.0000    0.0000    0.0000         2
           5     0.1667    1.0000    0.2857         2
           6     0.0000    0.0000    0.0000         2
           7     0.0000    0.0000    0.0000         2
           8     0.0000    0.0000    0.0000         2
           9     0.0667    0.5000    0.1176         2
          10     0.0000    0.0000    0.0000         2
          11     0.0000    0.0000    0.0000         2
          12     0.0000    0.0000    0.0000         2
          13     0.0000    0.0000    0.0000         2
          14     0.0000    0.0000    0.0000         2
          15     0.0000    0.0000    0.0000         2
          16     0.0000    0.0000    0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
print("Unique labels in Y_test:", np.unique(np.argmax(Y_test, axis=1)))

Unique labels in Y_test: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
